In [1]:
#Data Manipulation
import numpy as np
import pandas as pd

#Plotting and visualization
import matplotlib.pyplot as plt
import seaborn as sns #Some shiny visualizations

#Utilities
import json
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, f1_score, recall_score

#Artificial Neural Networks
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Activation, Dropout, BatchNormalization

Using TensorFlow backend.


In [3]:
#--------- Project Administration ------------
DatasetFilepath = './SLE_RA_Control_blood_panels.csv'
#--------------------------------------------

In [17]:
data = pd.read_csv(DatasetFilepath)

In [19]:
data

,Unnamed: 0,Name,Condition,LOC100287552,CDK4,C14orf178,LOC100130938,CCDC33,NDE1,RILPL2,...,CFHR5,CARTPT,HIST1H4J /// HIST1H4K,DNAJC1,LDHB,MTMR4,RGS22,LOC100288985,PAH,ZNF837
0,0,GSM1101456,disease: Rheumatoid Arthiritis (DMARD-IR),9.275148,7.992868,3.338075,4.957583,2.225178,9.736718,10.222316,...,2.224344,2.224344,5.345541,7.938424,12.422730,9.972145,2.224344,2.224344,2.224344,2.347913
1,1,GSM1101239,disease: Rheumatoid Arthiritis (TNF-IR),8.613589,6.788430,2.238413,6.861613,2.234680,8.729867,10.061225,...,2.234680,2.234680,5.519316,7.524966,11.807896,9.604994,2.239266,2.234680,2.234680,2.463903
2,2,GSM1101237,disease: Rheumatoid Arthiritis (TNF-IR),8.454822,6.208352,2.238413,6.831091,2.234680,10.119027,10.712793,...,2.234680,2.234680,5.853642,7.876886,10.971098,9.684565,2.239266,2.234680,2.234680,2.463903
3,3,GSM1101168,disease: Rheumatoid Arthiritis (TNF-IR),8.792935,6.924391,2.238413,5.885307,2.234680,8.771171,10.214039,...,2.234680,2.290698,5.052847,7.745385,11.864119,9.466255,2.239266,2.234680,2.234680,2.463903
4,4,GSM1101203,disease: Rheumatoid Arthiritis (TNF-IR),8.116888,7.331694,2.238413,6.490752,2.234680,9.186047,10.199805,...,2.234680,2.234680,5.519316,7.698570,12.206409,9.533367,2.239266,2.234680,2.234680,2.463903
5,5,GSM1101634,disease: Rheumatoid Arthiritis (DMARD-IR),8.305740,7.054037,2.318549,7.054975,2.378220,9.953732,10.088010,...,2.224344,2.224344,5.872254,7.942364,12.009564,8.881029,2.224344,2.224344,2.224344,2.394390
6,6,GSM1101241,disease: Rheumatoid Arthiritis (TNF-IR),9.036169,7.464291,2.238413,6.360375,2.234680,8.526447,10.330949,...,2.234680,2.234680,4.344519,7.816373,12.028614,9.851019,2.239266,2.234680,2.234680,2.438713
7,7,GSM1101276,disease: Rheumatoid Arthiritis (DMARD-IR),8.484262,7.041498,2.224344,4.472904,2.225178,9.460250,9.890115,...,2.224344,2.224344,4.697053,7.289914,12.319909,9.140982,2.224344,2.224344,2.224344,2.291556
8,8,GSM1101412,disease: Rheumatoid Arthiritis (DMARD-IR),8.740165,7.545408,2.224344,6.453334,2.225178,9.642805,9.906284,...,2.224344,2.275684,4.909019,7.948454,11.890901,8.932684,2.224344,2.224344,2.224344,2.249859
9,9,GSM1101164,disease: Rheumatoid Arthiritis (TNF-IR),8.635728,6.585438,2.238413,6.850843,2.234680,9.576137,10.268019,...,2.234680,2.234680,6.166206,7.707193,11.635475,9.240637,2.239266,2.234680,2.234680,2.463903


In [ ]:
data_corr = data.corr()
plt.figure(figsize=(40,40))
sns.heatmap(data_corr)
plt.show()

In [ ]:
x = data.drop('Name', axis=1)
x = x.drop('Exited', axis=1)

In [ ]:
# Prepare and scale the data. 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=25)

x_train = StandardScaler().fit_transform(x_train)
x_test = StandardScaler().fit_transform(x_test)

In [ ]:
#--------------------------------------------------------------------------
def make_that_model(layers, loss_function, optimizer, learning_rate, exit_activation=None):
    model = keras.Sequential()
    for i, layer in enumerate(layers):
        if i==0:
            model.add(getattr(keras.layers, layer["name"])(input_shape=INPUT_SHAPE, **layer["options"]))
        else:
            model.add(getattr(keras.layers, layer["name"])(**layer["options"]))

    if exit_activation!=None:
        model.add(Activation(exit_activation))

    op = getattr(keras.optimizers, optimizer)(learning_rate=(learning_rate))
    model.compile(loss=loss_function, optimizer=op, metrics=['accuracy'])

    return model
#--------------------------------------------------------------------------
def model_create(parameterization, load_file=None):

  model = make_that_model(
      parameterization.get('layers'),
      parameterization.get('loss_function'),
      parameterization.get('optimizer'),
      parameterization.get('learning_rate'),
      )
  
  if load_file != None:
    model.load_weights(load_file)
  
  return model
#--------------------------------------------------------------------------

In [ ]:
model = model_create(config)
model.summary()

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
      PROJECT_ROOT + "/Checkpoints/" + str(MODEL_COUNT) + '/run' + str(RUN_COUNT) + '.{epoch:02d}__L{loss:.2f}_A{accuracy:.2f}.hdf5',
      monitor="loss",
      save_best_only=True,
      save_weights_only=False,
      mode="min",
      save_freq="epoch",
      verbose=1,
  )

hist = model.fit(
        x=x_train, y=y_train, 
        steps_per_epoch=len(x_train), 
        validation_data=(x_test,y_test),
        validation_freq=1,
        workers=15,
        use_multiprocessing=True,
        epochs=EPOCHS, 
        callbacks=[checkpoint],
        verbose=1,
        )